### In Progress

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import folium # map rendering library
from functools import reduce
import scipy

print('Libraries imported.')

Libraries imported.


In [2]:
df1=pd.read_csv('Evictions.csv')

In [3]:
df1.head()

,COURT_INDEX_NUMBER,DOCKET_NUMBER,EVICTION_ADDRESS,EVICTION_APT_NUM,EXECUTED_DATE,MARSHAL_FIRST_NAME,MARSHAL_LAST_NAME,RESIDENTIAL_COMMERCIAL_IND,BOROUGH,EVICTION_ZIP
0,62712/18,19077,80-15 41ST AVENUE,215,09/20/2018,Edward,Guida,Residential,QUEENS,11373
1,54221/17,332335,31 EAST 31ST STREET,GROUND,09/19/2017,Thomas,Bia,Commercial,MANHATTAN,10016
2,B42362/19,117551,1605 METROPOLITAN AVENUE,9H,02/26/2020,Darlene,Barone,Residential,BRONX,10462
3,K85736/19,102339,1236 UNION STREET,3D,02/26/2020,Ileana,Rivera,Residential,BROOKLYN,11225
4,24576/17,2778,1085 NELSON AVE,1A,12/13/2017,Vadim,Barbarovich,Residential,BRONX,10452


Let's see the date range for the data.

In [4]:
df1['EXECUTED_DATE'].min()

'01/02/2018'

In [5]:
df1['EXECUTED_DATE'].max()

'12/31/2019'

This appears to be two years worth of data.

In [6]:
df1.head()

,COURT_INDEX_NUMBER,DOCKET_NUMBER,EVICTION_ADDRESS,EVICTION_APT_NUM,EXECUTED_DATE,MARSHAL_FIRST_NAME,MARSHAL_LAST_NAME,RESIDENTIAL_COMMERCIAL_IND,BOROUGH,EVICTION_ZIP
0,62712/18,19077,80-15 41ST AVENUE,215,09/20/2018,Edward,Guida,Residential,QUEENS,11373
1,54221/17,332335,31 EAST 31ST STREET,GROUND,09/19/2017,Thomas,Bia,Commercial,MANHATTAN,10016
2,B42362/19,117551,1605 METROPOLITAN AVENUE,9H,02/26/2020,Darlene,Barone,Residential,BRONX,10462
3,K85736/19,102339,1236 UNION STREET,3D,02/26/2020,Ileana,Rivera,Residential,BROOKLYN,11225
4,24576/17,2778,1085 NELSON AVE,1A,12/13/2017,Vadim,Barbarovich,Residential,BRONX,10452


In [7]:
df1['BOROUGH'].value_counts()

BRONX            22034
BROOKLYN         18800
QUEENS           13162
MANHATTAN        10159
STATEN ISLAND     2168
Name: BOROUGH, dtype: int64

In [8]:
df_zipcode=df1[['EVICTION_ZIP']]
df_zipcode.head()

,EVICTION_ZIP
0,11373
1,10016
2,10462
3,11225
4,10452


In [9]:
df_zipcode=pd.DataFrame(df_zipcode['EVICTION_ZIP'].value_counts())

In [10]:
df_zipcode.reset_index(inplace=True)
df_zipcode.rename(columns={'index':'Zip_Code','EVICTION_ZIP':'Counts'},inplace=True)
df_zipcode

,Zip_Code,Counts
0,10467,1835
1,10458,1799
2,10456,1742
3,10453,1667
4,10457,1462
...,...,...
218,11753,1
219,10152,1
220,11240,1
221,11024,1


In [11]:
df_zipcode.columns.values

array(['Zip_Code', 'Counts'], dtype=object)

In [12]:
df_zipcode

,Zip_Code,Counts
0,10467,1835
1,10458,1799
2,10456,1742
3,10453,1667
4,10457,1462
...,...,...
218,11753,1
219,10152,1
220,11240,1
221,11024,1


In [13]:
df_zipcode=df_zipcode[df_zipcode['Zip_Code']>9999]
df_zipcode['Latitude']=''
df_zipcode['Longitude']=''
df_zipcode.head()

/home/user-1131/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/user-1131/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Zip_Code,Counts,Latitude,Longitude
0,10467,1835,,
1,10458,1799,,
2,10456,1742,,
3,10453,1667,,
4,10457,1462,,


In [14]:
df_zipcode.shape

(221, 4)

Let's create a dataframe with zip code is NY state

In [15]:
us_zipcodes=pd.read_csv('us-zip-code-latitude-and-longitude.csv',sep=';')

In [16]:
us_zipcodes.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,71937,Cove,AR,34.398483,-94.39398,-6,1,"34.398483,-94.39398"
1,72044,Edgemont,AR,35.624351,-92.16056,-6,1,"35.624351,-92.16056"
2,56171,Sherburn,MN,43.660847,-94.74357,-6,1,"43.660847,-94.74357"
3,49430,Lamont,MI,43.010337,-85.89754,-5,1,"43.010337,-85.89754"
4,52585,Richland,IA,41.194129,-91.98027,-6,1,"41.194129,-91.98027"


In [17]:
ny_state_zip=us_zipcodes[us_zipcodes['State']=='NY']

In [18]:
ny_state_zip.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
21,14043,Depew,NY,42.904958,-78.700600,-5,1,"42.904958,-78.7006"
72,13827,Owego,NY,42.120274,-76.258740,-5,1,"42.120274,-76.25874"
73,14739,Friendship,NY,42.192384,-78.143080,-5,1,"42.192384,-78.14308"
85,11703,North Babylon,NY,40.733398,-73.322570,-5,1,"40.733398,-73.32257"
131,11707,West Babylon,NY,40.922326,-72.637078,-5,1,"40.922326,-72.637078"


In [19]:
ny_state_zip.columns.values

array(['Zip', 'City', 'State', 'Latitude', 'Longitude', 'Timezone',
       'Daylight savings time flag', 'geopoint'], dtype=object)

In [20]:
ny_state_zip.rename(columns={'Zip':'Zip_Code'},inplace=True)
ny_state_zip

/home/user-1131/venv/lib/python3.6/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,Zip_Code,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
21,14043,Depew,NY,42.904958,-78.700600,-5,1,"42.904958,-78.7006"
72,13827,Owego,NY,42.120274,-76.258740,-5,1,"42.120274,-76.25874"
73,14739,Friendship,NY,42.192384,-78.143080,-5,1,"42.192384,-78.14308"
85,11703,North Babylon,NY,40.733398,-73.322570,-5,1,"40.733398,-73.32257"
131,11707,West Babylon,NY,40.922326,-72.637078,-5,1,"40.922326,-72.637078"
...,...,...,...,...,...,...,...,...
43033,11507,Albertson,NY,40.769942,-73.650040,-5,1,"40.769942,-73.65004"
43139,12979,Rouses Point,NY,44.990606,-73.370170,-5,1,"44.990606,-73.37017"
43140,13452,Saint Johnsville,NY,43.021223,-74.645370,-5,1,"43.021223,-74.64537"
43154,10211,New York,NY,40.780751,-73.977182,-5,1,"40.780751,-73.977182"


The next step is to put together df_zipcode with ny_state_zip based on zip code.

In [21]:
df_combined=pd.merge(df_zipcode,ny_state_zip,on='Zip_Code',how='left')

In [22]:
df_combined.drop('Latitude_x',axis=1,inplace=True)

In [23]:
df_combined.drop('Longitude_x',axis=1,inplace=True)

In [24]:
df_combined.dropna(inplace=True)
df_combined

,Zip_Code,Counts,City,State,Latitude_y,Longitude_y,Timezone,Daylight savings time flag,geopoint
0,10467,1835,Bronx,NY,40.872265,-73.869370,-5.0,1.0,"40.872265,-73.86937"
1,10458,1799,Bronx,NY,40.864166,-73.888810,-5.0,1.0,"40.864166,-73.88881"
2,10456,1742,Bronx,NY,40.829676,-73.908560,-5.0,1.0,"40.829676,-73.90856"
3,10453,1667,Bronx,NY,40.853017,-73.912140,-5.0,1.0,"40.853017,-73.91214"
4,10457,1462,Bronx,NY,40.846745,-73.898610,-5.0,1.0,"40.846745,-73.89861"
...,...,...,...,...,...,...,...,...,...
216,11753,1,Jericho,NY,40.786544,-73.536750,-5.0,1.0,"40.786544,-73.53675"
217,10152,1,New York,NY,40.758611,-73.972230,-5.0,1.0,"40.758611,-73.97223"
218,11240,1,Brooklyn,NY,40.645099,-73.945032,-5.0,1.0,"40.645099,-73.945032"
219,11024,1,Great Neck,NY,40.812887,-73.740940,-5.0,1.0,"40.812887,-73.74094"


In [25]:
df_combined.rename(columns={'Latitude_y':'Latitude','Longitude_y':'Longitude'},inplace=True)

Let's save a copy of df_combined, just in case.

In [26]:
df_combined.to_csv('combined_table.csv',index=0)

In [27]:
df_combined=pd.read_csv('combined_table.csv')
df_combined

,Zip_Code,Counts,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,10467,1835,Bronx,NY,40.872265,-73.869370,-5.0,1.0,"40.872265,-73.86937"
1,10458,1799,Bronx,NY,40.864166,-73.888810,-5.0,1.0,"40.864166,-73.88881"
2,10456,1742,Bronx,NY,40.829676,-73.908560,-5.0,1.0,"40.829676,-73.90856"
3,10453,1667,Bronx,NY,40.853017,-73.912140,-5.0,1.0,"40.853017,-73.91214"
4,10457,1462,Bronx,NY,40.846745,-73.898610,-5.0,1.0,"40.846745,-73.89861"
...,...,...,...,...,...,...,...,...,...
198,11753,1,Jericho,NY,40.786544,-73.536750,-5.0,1.0,"40.786544,-73.53675"
199,10152,1,New York,NY,40.758611,-73.972230,-5.0,1.0,"40.758611,-73.97223"
200,11240,1,Brooklyn,NY,40.645099,-73.945032,-5.0,1.0,"40.645099,-73.945032"
201,11024,1,Great Neck,NY,40.812887,-73.740940,-5.0,1.0,"40.812887,-73.74094"


In [28]:
df_combined.dtypes

Zip_Code                        int64
Counts                          int64
City                           object
State                          object
Latitude                      float64
Longitude                     float64
Timezone                      float64
Daylight savings time flag    float64
geopoint                       object
dtype: object

In [29]:
df_combined.shape

(203, 9)

In [30]:
city='New York City, New York, United States of America'
geolocator=Nominatim(user_agent="nyc_explorer")
location=geolocator.geocode(city)
lat=location.latitude
lon=location.longitude
lat,lon

(40.7127281, -74.0060152)

In [31]:
NYC_map=folium.Map(location=[lat-0.1,lon],zoom_start=10)        #0.1 is the offset to center the map.
NYC_map

We'll mark the latitude and longitude of each zip code and include eviction counts in the label.

In [32]:
for lat, lon, zipcode,city,counts in zip(df_combined['Latitude'], df_combined['Longitude'], df_combined['Zip_Code'],df_combined['City'],df_combined['Counts']):
    label = 'City: {}, Zipcode: {}, Counts: {}'.format(city,zipcode,counts)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lon],
    radius=4,
    popup=label,
    color='orange',
    line_width=1,
    fill=True,
    fill_color='red',
    fill_opacity=0.5,
    parse_html=False).add_to(NYC_map)  
    
NYC_map

We cluster the markers to make the map more legible. Hovering mouse over the clusters show the number of markers in it. When we zoom in to individual markers. Hovering over individual markers show the number of evictions.

In [33]:
#Group markers into different clusters

from folium import plugins #import plugins

latitude=location.latitude
longitude=location.longitude

# let's start again with a clean copy of the map of San Francisco
NYC_map = folium.Map(location = [latitude, longitude], zoom_start = 9)

# instantiate a mark cluster object for the incidents in the dataframe
evictions = plugins.MarkerCluster().add_to(NYC_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(df_combined.Latitude, df_combined.Longitude, df_combined.Counts):
    folium.Marker(
        location=[lat, lng],
        icon=None,
        popup=label,
    ).add_to(evictions)

# display map
NYC_map

In our next step we'll delve deeper into the residential evictions. Let's take a look at our original dataframe df1. For illustration purpose, this time we'll go over the process again, except we'll use Nominatim to extract coordinates.

In [34]:
df1.head()

,COURT_INDEX_NUMBER,DOCKET_NUMBER,EVICTION_ADDRESS,EVICTION_APT_NUM,EXECUTED_DATE,MARSHAL_FIRST_NAME,MARSHAL_LAST_NAME,RESIDENTIAL_COMMERCIAL_IND,BOROUGH,EVICTION_ZIP
0,62712/18,19077,80-15 41ST AVENUE,215,09/20/2018,Edward,Guida,Residential,QUEENS,11373
1,54221/17,332335,31 EAST 31ST STREET,GROUND,09/19/2017,Thomas,Bia,Commercial,MANHATTAN,10016
2,B42362/19,117551,1605 METROPOLITAN AVENUE,9H,02/26/2020,Darlene,Barone,Residential,BRONX,10462
3,K85736/19,102339,1236 UNION STREET,3D,02/26/2020,Ileana,Rivera,Residential,BROOKLYN,11225
4,24576/17,2778,1085 NELSON AVE,1A,12/13/2017,Vadim,Barbarovich,Residential,BRONX,10452


In [35]:
df_evictions_residential=df1[df1['RESIDENTIAL_COMMERCIAL_IND']=='Residential']
df_evictions_residential

,COURT_INDEX_NUMBER,DOCKET_NUMBER,EVICTION_ADDRESS,EVICTION_APT_NUM,EXECUTED_DATE,MARSHAL_FIRST_NAME,MARSHAL_LAST_NAME,RESIDENTIAL_COMMERCIAL_IND,BOROUGH,EVICTION_ZIP
0,62712/18,19077,80-15 41ST AVENUE,215,09/20/2018,Edward,Guida,Residential,QUEENS,11373
2,B42362/19,117551,1605 METROPOLITAN AVENUE,9H,02/26/2020,Darlene,Barone,Residential,BRONX,10462
3,K85736/19,102339,1236 UNION STREET,3D,02/26/2020,Ileana,Rivera,Residential,BROOKLYN,11225
4,24576/17,2778,1085 NELSON AVE,1A,12/13/2017,Vadim,Barbarovich,Residential,BRONX,10452
6,B55306/17,101402,1129 ST. LAWRENCE AVENUE,4D,12/19/2018,Darlene,Barone,Residential,BRONX,10472
...,...,...,...,...,...,...,...,...,...,...
66317,63498/17,78997,2333 WEBSTER AVENUE,3L,02/15/2018,Henry,Daley,Residential,BRONX,10458
66318,N77591/16,76847,408 EAST 64TH STREET,2B,05/17/2017,Ileana,Rivera,Residential,MANHATTAN,10065
66320,68649/17,73953,712 REMSEN AVENUE,NaN,09/15/2017,Henry,Daley,Residential,BROOKLYN,11236
66321,77370/18,22034,110-19 FRANCIS LEWIS BLVD.,NaN,05/28/2019,Edward,Guida,Residential,QUEENS,11429


In [36]:
df_evictions_residential.columns.values

array(['COURT_INDEX_NUMBER', 'DOCKET_NUMBER', 'EVICTION_ADDRESS',
       'EVICTION_APT_NUM', 'EXECUTED_DATE', 'MARSHAL_FIRST_NAME',
       'MARSHAL_LAST_NAME', 'RESIDENTIAL_COMMERCIAL_IND', 'BOROUGH',
       'EVICTION_ZIP'], dtype=object)

We'll drop all the columns except 'EVICTION_ADDRESS', 'EXECUTED_DATE', 'RESIDENTIAL_COMMERCIAL_IND', 'BOROUGH' and 'EVICTION_ZIP'.

In [37]:
df_evictions_residential.drop('COURT_INDEX_NUMBER',axis=1,inplace=True)
df_evictions_residential.drop('DOCKET_NUMBER',axis=1,inplace=True)
df_evictions_residential.drop('EVICTION_APT_NUM',axis=1,inplace=True)
df_evictions_residential.drop('MARSHAL_FIRST_NAME',axis=1,inplace=True)
df_evictions_residential.drop('MARSHAL_LAST_NAME',axis=1,inplace=True)

df_evictions_residential.head()

/home/user-1131/venv/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,EVICTION_ADDRESS,EXECUTED_DATE,RESIDENTIAL_COMMERCIAL_IND,BOROUGH,EVICTION_ZIP
0,80-15 41ST AVENUE,09/20/2018,Residential,QUEENS,11373
2,1605 METROPOLITAN AVENUE,02/26/2020,Residential,BRONX,10462
3,1236 UNION STREET,02/26/2020,Residential,BROOKLYN,11225
4,1085 NELSON AVE,12/13/2017,Residential,BRONX,10452
6,1129 ST. LAWRENCE AVENUE,12/19/2018,Residential,BRONX,10472


Next we remove all rows with NaNs.

In [38]:
df_evictions_residential.dropna(inplace=True)

/home/user-1131/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Let's take a look at the number of rows and columns.

In [39]:
df_evictions_residential.shape

(60747, 5)

In [40]:
df_evictions_residential

,EVICTION_ADDRESS,EXECUTED_DATE,RESIDENTIAL_COMMERCIAL_IND,BOROUGH,EVICTION_ZIP
0,80-15 41ST AVENUE,09/20/2018,Residential,QUEENS,11373
2,1605 METROPOLITAN AVENUE,02/26/2020,Residential,BRONX,10462
3,1236 UNION STREET,02/26/2020,Residential,BROOKLYN,11225
4,1085 NELSON AVE,12/13/2017,Residential,BRONX,10452
6,1129 ST. LAWRENCE AVENUE,12/19/2018,Residential,BRONX,10472
...,...,...,...,...,...
66317,2333 WEBSTER AVENUE,02/15/2018,Residential,BRONX,10458
66318,408 EAST 64TH STREET,05/17/2017,Residential,MANHATTAN,10065
66320,712 REMSEN AVENUE,09/15/2017,Residential,BROOKLYN,11236
66321,110-19 FRANCIS LEWIS BLVD.,05/28/2019,Residential,QUEENS,11429


In [41]:
df3=df_evictions_residential

To look up using Nominatim package, we need a complete address. To have that handy we add a new column named 'LOOKUP ADDRESS'.

In [47]:
df3

,EVICTION_ADDRESS,EXECUTED_DATE,RESIDENTIAL_COMMERCIAL_IND,BOROUGH,EVICTION_ZIP
0,80-15 41ST AVENUE,09/20/2018,Residential,QUEENS,11373
2,1605 METROPOLITAN AVENUE,02/26/2020,Residential,BRONX,10462
3,1236 UNION STREET,02/26/2020,Residential,BROOKLYN,11225
4,1085 NELSON AVE,12/13/2017,Residential,BRONX,10452
6,1129 ST. LAWRENCE AVENUE,12/19/2018,Residential,BRONX,10472
...,...,...,...,...,...
66317,2333 WEBSTER AVENUE,02/15/2018,Residential,BRONX,10458
66318,408 EAST 64TH STREET,05/17/2017,Residential,MANHATTAN,10065
66320,712 REMSEN AVENUE,09/15/2017,Residential,BROOKLYN,11236
66321,110-19 FRANCIS LEWIS BLVD.,05/28/2019,Residential,QUEENS,11429


In [48]:
df3['FULL_ADDRESS']=''
df3

/home/user-1131/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,EVICTION_ADDRESS,EXECUTED_DATE,RESIDENTIAL_COMMERCIAL_IND,BOROUGH,EVICTION_ZIP,FULL_ADDRESS
0,80-15 41ST AVENUE,09/20/2018,Residential,QUEENS,11373,
2,1605 METROPOLITAN AVENUE,02/26/2020,Residential,BRONX,10462,
3,1236 UNION STREET,02/26/2020,Residential,BROOKLYN,11225,
4,1085 NELSON AVE,12/13/2017,Residential,BRONX,10452,
6,1129 ST. LAWRENCE AVENUE,12/19/2018,Residential,BRONX,10472,
...,...,...,...,...,...,...
66317,2333 WEBSTER AVENUE,02/15/2018,Residential,BRONX,10458,
66318,408 EAST 64TH STREET,05/17/2017,Residential,MANHATTAN,10065,
66320,712 REMSEN AVENUE,09/15/2017,Residential,BROOKLYN,11236,
66321,110-19 FRANCIS LEWIS BLVD.,05/28/2019,Residential,QUEENS,11429,


In [61]:
pd.DataFrame.to_string(df3[['EVICTION_ZIP']])
type(df3[['EVICTION_ZIP']])

TypeError: to_string() got an unexpected keyword argument 'inplace'

In [60]:
df3.dtypes

EVICTION_ADDRESS              object
EXECUTED_DATE                 object
RESIDENTIAL_COMMERCIAL_IND    object
BOROUGH                       object
EVICTION_ZIP                   int64
FULL_ADDRESS                  object
dtype: object